In [4]:
from pathlib import Path
from PIL import Image
import pytesseract
import cv2
import os
import csv

from spellchecker import SpellChecker
import re


In [ ]:

def preprocess_image(image_path, output_path):
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # med_blur = cv2.GaussianBlur(gray,(5,5),0)

    denoised = cv2.fastNlMeansDenoising(gray,None, 10, 7, 21)
    # denoised = cv2.GaussianBlur(gray, (3, 3), 0)  # smaller the kernel size
    # denoised = cv2.bilateralFilter(gray, 9, 75, 75)  # better edges

#     coordinates = cv2.findNonZero(denoised)
#     if coordinates is not None:
#         angle =cv2.minAreaRect(coordinates)[-1]
#         if angle < -45:
#             angle = -(90+angle)
#         else:
#             angle = -angle

#         # Get the center of the image for rotation
#         (h, w) = denoised.shape[:2]
#         center = (w // 2, h // 2)

#         # Compute the rotation matrix
#         matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
#  # Rotate the image to deskew it
#         denoised = cv2.warpAffine(denoised, matrix, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    # binary = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2) #This method calculates the threshold for a pixel using a weighted sum (Gaussian distribution) of the pixel values in its neighborhood, minus a constant. It’s effective when the image has non-uniform lighting.
    # binary = cv2.adaptiveThreshold(denoised, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY, 11, 2)
    # _, binary = cv2.threshold(denoised, 150, 255, cv2.THRESH_BINARY) #Global thresholding
    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    height, width = binary.shape
    # new_size = (int(width * 1.5), int(height * 1.5))  
    # resized = cv2.resize(binary, new_size, interpolation=cv2.INTER_LINEAR)
    # resized = cv2.resize(binary, (int(width / 2), int(height / 2)), interpolation=cv2.INTER_LINEAR)
    resized = cv2.resize(binary, (int(width * 2), int(height * 2)), interpolation=cv2.INTER_CUBIC)

    pil_image = Image.fromarray(resized)
    
    pil_image.save(output_path) # ,dpi=(300, 300)

    # Save the processed image
    # cv2.imwrite(output_path, resized)

input_folder = '../recipe_pics/recipes'
output_folder = '../recipe_pics_processed'

for filename in os.listdir(input_folder):
    if filename.endswith('.jpg') or filename.endswith('.jpeg'):
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'processed_{filename}')
        preprocess_image(input_path, output_path)


In [ ]:

def preprocess_image(image_path, output_path):
    image = cv2.imread(image_path)

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    denoised = cv2.fastNlMeansDenoising(gray, None, 10, 7, 21)

    _, binary = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    height, width = binary.shape
    resized = cv2.resize(binary, (int(width * 2), int(height * 2)), interpolation=cv2.INTER_CUBIC)

    pil_image = Image.fromarray(resized)
    pil_image.save(output_path, dpi=(300, 300))  # Adjust DPI as necessary

input_folder = '../recipe_pics'
output_folder = '../recipe_pics_processed'

os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg')):  # Case-insensitive check for .jpg and .jpeg
        input_path = os.path.join(input_folder, filename)
        output_path = os.path.join(output_folder, f'processed_{filename}')
        preprocess_image(input_path, output_path)

print("Image preprocessing completed!")


Image preprocessing completed!


In [ ]:
dic = '../recipe_pics_processed'


In [4]:

files = Path(dic).glob('*jpg')

custom_config =  r'--oem 1 --psm 6'

for file in files:
    print(file)
    print(pytesseract.image_to_string(Image.open(file)))
    print('\n---------------\n')

C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_0027c0d2be4bdf38ed30e2cb258f3dc9.jpg


---------------

C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_005025fb1e3c6a8d55d55aa639a7d16b.jpg


---------------

C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_00c0eaa6abb96cb5ab275976f2c7eb60.jpg
HEARTY BEEF AND VEGETABLE SOUP

| pAb tethering hi tm CD re cme ones

Vance plecee Gobech. WNie. raed] | eee;
1 het ete, ire 24909 grid baney
40a wene oe ere
ES unlindaalrmnsit ition’

Bape Biee pananan 19 hirwe 1 settee set
Dev MHd HO  et) —| heee nage

1 ng nt tery 12 ra PD) LF mega Pape

Pace 95 ay CIN Seng bee stores Tess Me gh ra Heed
ivr he boing tom Mer in emnte | ate?

Se Cott emt het E
tithe aes Rare batlerna ro Apart Sat he beat ve ale
ne. aed bene chergng Me are ho wien

ig pov Sono 0 Lembongan rere arad HNO
Yew a5 be aphid ott Pew mee Foret pe tnt



---------------

C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\pro

In [ ]:
from pathlib import Path
from PIL import Image
import pytesseract
import re

def parse_recipe_tesseract(text):

    title, ingredients, instructions = [], [], []
    lines = text.split('\n')
    in_ingredients, in_instructions = False, False

    for line in lines:
        line = line.strip()
        if not line:
            continue
        if re.search(r'\b(ingredients|items)\b', line, re.IGNORECASE):
            in_ingredients, in_instructions = True, False
            continue
        if re.search(r'\b(instructions|how to|directions|method)\b', line, re.IGNORECASE):
            in_instructions, in_ingredients = True, False
            continue
        if not in_ingredients and not in_instructions and len(title) < 3:
            title.append(line)
        if in_ingredients:
            ingredients.append(line)
        if in_instructions:
            instructions.append(line)
    return {
        'title': ' '.join(title).strip() if title else "N/A",
        'ingredients': [item.strip() for item in ingredients] if ingredients else ["N/A"],
        'instructions': ' '.join([item.strip() for item in instructions]) if instructions else "N/A"
    }

def process_recipes_tesseract(image_folder, custom_config, output_csv_path):
    files = Path(image_folder).glob('*jpg')
    parsed_recipes = []
    
    for file in files:
        print(f"Processing {file}...")
        ocr_text = pytesseract.image_to_string(Image.open(file), config=custom_config)
        print(f"OCR Output for {file}:\n{ocr_text}\n")
        parsed_recipe = parse_recipe_tesseract(ocr_text)
        print("Parsed Recipe:")
        print(f"Title: {parsed_recipe['title']}")
        print(f"Ingredients: {', '.join(parsed_recipe['ingredients'])}")
        print(f"Instructions: {parsed_recipe['instructions']}")
        print('\n---------------\n')
        parsed_recipes.append(parsed_recipe)
    
    with open(output_csv_path, 'w', encoding='utf-8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=['title', 'ingredients', 'instructions'])
        writer.writeheader()
        for recipe in parsed_recipes:
            writer.writerow({
                'title': recipe['title'],
                'ingredients': '; '.join(parsed_recipe['ingredients']),
                'instructions': parsed_recipe['instructions']
            })
    print(f"\nAll recipes have been saved to {output_csv_path}")

image_folder = '../recipe_pics_processed'

output_csv_path = '../new_parsed_recipes_tesseract.csv'

custom_config = r'--oem 1 --psm 6'

process_recipes_tesseract(image_folder, custom_config, output_csv_path)


Processing C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_0027c0d2be4bdf38ed30e2cb258f3dc9.jpg...
OCR Output for C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_0027c0d2be4bdf38ed30e2cb258f3dc9.jpg:
proccess QA.
CHOCOLATE )
Crip COOKIES (
Ps) \e
@-—— 6
e ° :
e “. . “ °
ee °
e @


Parsed Recipe:
Title: proccess QA. CHOCOLATE ) Crip COOKIES (
Ingredients: N/A
Instructions: N/A

---------------

Processing C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_005025fb1e3c6a8d55d55aa639a7d16b.jpg...
OCR Output for C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_005025fb1e3c6a8d55d55aa639a7d16b.jpg:
FOCIE FRIDAY : 4it i
re


Parsed Recipe:
Title: FOCIE FRIDAY : 4it i re
Ingredients: N/A
Instructions: N/A

---------------

Processing C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\processed_00c0eaa6abb96cb5ab275976f2c7eb60.jpg...
OCR Output for C:\Users\oyku_\Desktop\image-pro\im\recipe_pics_processed\proces

In [ ]:
def calculate_spelling_accuracy(text_file):

    spell = SpellChecker()

    with open(text_file, 'r', encoding='utf-8') as f:
        text_content = f.read()

    words = text_content.split()

    misspelled = spell.unknown(words)

    total_words = len(words)
    correct_words = total_words - len(misspelled)
    accuracy = correct_words / total_words if total_words > 0 else 0

    print(f"Total Words: {total_words}")
    print(f"Correct Words: {correct_words}")
    print(f"Misspelled Words: {len(misspelled)}")
    print(f"Spelling Accuracy: {accuracy:.2%}")

    return accuracy

text_file = '../new_parsed_recipes_tesseract.csv'

calculate_spelling_accuracy(text_file)

Total Words: 5740
Correct Words: 3335
Misspelled Words: 2405
Spelling Accuracy: 58.10%


0.5810104529616724

: 